In [1]:
import psycopg2
from psycopg2 import OperationalError, InterfaceError
from google import genai
from google.genai import types
from google.genai.types import Part, UserContent
from utils import *
import logging 
from log_config import configure_logging

In [ ]:
MODEL_GENAI = "gemini-2.0-flash"
GENAI_API_KEY = "" 

DATABASE_CONFIG = {
    "dbname": "tensorflow_data",
    "user": "postgres",
    "password": "",
    "host": "",
    "port": 5432,
    "keepalives":          1,
    "keepalives_idle":     30,  # envia primeiro ping após 30s
    "keepalives_interval": 10,  # intervalo entre pings
    "keepalives_count":    5,   # número de falhas antes de considerar caída
}

configure_logging()

In [3]:
def get_connection():
    return psycopg2.connect(**DATABASE_CONFIG)

In [4]:
def ensure_connection(conn):
    try:
        conn.cursor().execute("SELECT 1;")
    except (OperationalError, InterfaceError):
        logging.warning("Conexão perdida. Reconectando...")
        conn = get_connection()
    return conn

In [5]:
conn = get_connection()
cursor = conn.cursor()

In [6]:
cursor.execute(
    """
    SELECT 
        issue_id,
        title,
        body
    FROM   
        issues_from_release
    WHERE
        release_number LIKE '%2.17%'
    """
)
issues = cursor.fetchall()
len(issues)

405

In [7]:
client = genai.Client(api_key=GENAI_API_KEY)

In [8]:
def classify_comment_sentiment_genai(comment_body, chat_session):
    prompt = f"Classify the sentiment of this comment: {comment_body}"

    response = chat_session.send_message(prompt)
    
    comment_sentiment = response.text.lower()

    logging.info(f"Sentiment of the comment is: {comment_sentiment}")

    return comment_sentiment


In [9]:
def analyze_issue_sentiment_genai(issue_title, issue_body, comments, client, model):
    issue_chat_session = client.chats.create(
        model=model,
        history=[
            UserContent(parts=[Part(text=f"The title of the issue is: {issue_title}")]),
            UserContent(parts=[Part(text=f"The body of the issue is: {issue_body}")])
        ],
        config=types.GenerateContentConfig(
            temperature=0.2,
            system_instruction=(
                "You are a GitHub issue sentiment analysis expert. "
                "Classify each comment's sentiment as 'positive', 'negative', or 'neutral'. "
                "Answer with one word only."
            )
        )
    )

    results = []
    for comment in comments:
        comment_id, _, comment_body = comment

        logging.info(f"Classifying comment {comment_id}...")

        comment_sentiment = classify_comment_sentiment_genai(comment_body, issue_chat_session)

        results.append((comment_id, comment_sentiment))

    return results

In [10]:
for issue in issues:
    issue_id, issue_title, issue_body = issue

    # Mantém conexão viva antes de cada bloco
    conn = ensure_connection(conn)
    cursor = conn.cursor()

    # Getting the data
    try:
        comments = get_comments_by_issue_id(issue_id, cursor)
        logging.info(f"{len(comments)} comments found for issue {issue_id}.")
    except Exception as e:
        logging.error(f"Error when getting the comments for issue {issue_id}: {e}")
        continue

    # Analyzing sentiments
    try:
        logging.info(f"Starting sentiment analysis with Gemini 2.0 Flash for issue {issue_id}")
        sentiments_gemini = analyze_issue_sentiment_genai(
            issue_title=issue_title,
            issue_body=issue_body,
            comments=comments,
            client=client,
            model=MODEL_GENAI
        )
        logging.info(f"Sentiment analysis with Gemini 2.0 Flash concluded for issue {issue_id}.")
    except Exception as e:
        logging.error(f"Error when analyzing issue {issue_id} with Gemini 2.0 Flash.")
        continue

    # Saving the data
    try:
        save_sentiments_gemini(sentiments_gemini, cursor)
        conn.commit()
        logging.info(f"Data successfully saved for issue {issue_id}.")
    except Exception as e:
        logging.error(f"Error when saving the data for issue {issue_id}.")
        continue
    
cursor.close()  
conn.close() 

2025-05-08 14:10:15,840 - INFO - 12 comments found for issue 2719667595.
2025-05-08 14:10:15,841 - INFO - Starting sentiment analysis with Gemini 2.0 Flash for issue 2719667595
2025-05-08 14:10:15,842 - INFO - Classifying comment 2519970094...
2025-05-08 14:10:15,843 - INFO - AFC is enabled with max remote calls: 10.
2025-05-08 14:10:16,752 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 OK"
2025-05-08 14:10:16,781 - INFO - AFC remote call 1 is done.
2025-05-08 14:10:16,782 - INFO - Sentiment of the comment is: neutral

2025-05-08 14:10:16,783 - INFO - Classifying comment 2521726106...
2025-05-08 14:10:16,784 - INFO - AFC is enabled with max remote calls: 10.
2025-05-08 14:10:17,440 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 OK"
2025-05-08 14:10:17,442 - INFO - AFC remote call 1 is done.
2025-05-08 14:10:17,443 - INFO